In [6]:
import sys
sys.path.append('/mnt/d/hust/code/thesis')
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import current_timestamp, date_format
from spark_processing.pipeline.drop_null_columns import drop_null_columns
from spark_processing.pipeline.flatten_dataframe import flatten_dataframe
from spark_processing.pipeline.upsert_normalized_table import upsert_normalized_table
from delta import *
from spark_processing.pipeline.tgdd.udf import *

def upsert_by_categories(ingest_id):
    builder = SparkSession.builder \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    normalized_daily_table = f"/mnt/d/hust/code/thesis/warehouse/normalized/tgdd/{ingest_id}"
    df = spark.read.format("delta").load(normalized_daily_table)
    categories_df = df.select("category").distinct()
    category_list = [row['category'] for row in categories_df.collect()]
    for category in category_list:
        new_df = df.filter(F.col("category") == category)
        new_df = flatten_dataframe(new_df)
        new_df = drop_null_columns(new_df)
        new_df = new_df.withColumn("updated_at", date_format(current_timestamp(), 'yyyy-MM-dd HH:mm:ss'))
        normalized_table_path = f"/mnt/d/hust/code/thesis/warehouse/categories/tgdd/{category}"
        upsert_normalized_table(spark, new_df, normalized_table_path, 'product_id', 'product_id')
if __name__ == "__main__":
    # ingest_id = sys.argv[1]
    # upsert_by_categories(ingest_id)
    upsert_by_categories('20250305')


['/home/bien/.pyenv/versions/3.9.20/lib/python39.zip', '/tmp/spark-18b4fcff-0668-4fee-bdca-ab90124bbc9c/userFiles-9abab004-a4c2-4f41-893f-fd04efdf8fe7/org.antlr_antlr4-runtime-4.9.3.jar', '/tmp/spark-18b4fcff-0668-4fee-bdca-ab90124bbc9c/userFiles-9abab004-a4c2-4f41-893f-fd04efdf8fe7/io.delta_delta-storage-3.0.0.jar', '/tmp/spark-18b4fcff-0668-4fee-bdca-ab90124bbc9c/userFiles-9abab004-a4c2-4f41-893f-fd04efdf8fe7/io.delta_delta-spark_2.12-3.0.0.jar', '/tmp/spark-18b4fcff-0668-4fee-bdca-ab90124bbc9c/userFiles-9abab004-a4c2-4f41-893f-fd04efdf8fe7', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9/lib-dynload', '', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9/site-packages', '/mnt/d/hust/code/thesis/spark_processing/pipeline/tgdd', '/mnt/d/hust/code/thesis/spark_processing/pipeline/tgdd', '/mnt/d/hust/code/thesis/spark_processing/pipeline/tgdd', '/mnt/d/hust/code/thesis/spark_processing/pipeline/tgdd', '/mnt/d/hust/code/thesis', '/mnt/d

25/03/06 02:17:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/home/bien/.pyenv/versions/3.9.20/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/bien/.pyenv/versions/3.9.20/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/bien/.pyenv/versions/3.9.20/lib/python3.9/socket.py", line 716, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 